In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Player table

In [4]:
csv_file = "Resources/master_player.csv"
player_df = pd.read_csv(csv_file)
player_df.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,220.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,NaN,NaN,NaN,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


# Hall of Fame table

In [15]:
csv_file = "Resources/Hall_of_Fame.csv"
hof_df = pd.read_csv(csv_file)
hof_df.head()

,playerID,yearid,votedBy,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226,170,222,Y,Player,NaN
1,ruthba01,1936,BBWAA,226,170,215,Y,Player,NaN
2,wagneho01,1936,BBWAA,226,170,215,Y,Player,NaN
3,mathech01,1936,BBWAA,226,170,205,Y,Player,NaN
4,johnswa01,1936,BBWAA,226,170,189,Y,Player,NaN


### Clean up the hof table to only include inductee's 

In [33]:
# Only include playerID, yearid, category where inducted = Y
# Cleaned up version will go into hof_inductees table

new_hof_df = hof_df[['playerID','yearid','category']].where(hof_df['inducted']=='Y').dropna()
new_hof_df.head()


,playerID,yearid,category
0,cobbty01,1936,Player
1,ruthba01,1936,Player
2,wagneho01,1936,Player
3,mathech01,1936,Player
4,johnswa01,1936,Player


# Batting Stats table

In [42]:
csv_file = "Resources/Batting.csv"
batting_df = pd.read_csv(csv_file)
batting_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [41]:
# Only include primary stats of hof inductees
# Get a list of hof playerID's
hof_player_list = new_hof_df["playerID"].tolist()

# New Batting dataframe with only hof inductees
batting2_df = batting_df[['playerID','yearID','teamID','G','AB','R','H','2B','3B','HR','RBI','SB']]
boolean_series = batting2_df.playerID.isin(hof_player_list)
hof_batting_df = batting2_df[boolean_series]
hof_batting_df.head()

,playerID,yearID,teamID,G,AB,R,H,2B,3B,HR,RBI,SB
4,ansonca01,1871,RC1,25,120,29,39,11,3,0,16.0,6.0
98,spaldal01,1871,BS1,31,144,43,39,10,1,1,31.0,2.0
106,whitede01,1871,CL1,29,146,40,47,6,5,1,21.0,2.0
111,wrighge01,1871,BS1,16,80,33,33,7,5,0,11.0,9.0
112,wrighha01,1871,BS1,31,147,42,44,5,2,0,26.0,7.0


# Pitching Stats table

In [7]:
csv_file = "Resources/Pitching.csv"
pitching_df = pd.read_csv(csv_file)
pitching_df.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,...,NaN,7,NaN,0,146.0,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,...,NaN,7,NaN,0,1291.0,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,...,NaN,2,NaN,0,14.0,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,...,NaN,20,NaN,0,1080.0,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,...,NaN,0,NaN,0,57.0,0,21,NaN,NaN,NaN


# Player Awards table

In [8]:
csv_file = "Resources/AwardsPlayers.csv"
player_awards_df = pd.read_csv(csv_file)
player_awards_df.head()

,playerID,awardID,yearID,lgID,tie,notes
0,bondto01,Pitching Triple Crown,1877,NL,NaN,NaN
1,hinespa01,Triple Crown,1878,NL,NaN,NaN
2,heckegu01,Pitching Triple Crown,1884,AA,NaN,NaN
3,radboch01,Pitching Triple Crown,1884,NL,NaN,NaN
4,oneilti01,Triple Crown,1887,AA,NaN,NaN


## Connect to local database

In [29]:
rds_connection_string = "postgres:password@localhost:5432/mlb"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [30]:
engine.table_names()

['hof', 'people', 'awards', 'pitching', 'batting', 'hof_inductees']

In [12]:
player_df.to_sql(name='people', con=engine, if_exists='append', index=False)

In [31]:
new_hof_df.to_sql(name='hof_inductees', con=engine, if_exists='append', index=False)